# <span style = 'color: Coral'> **NYC Airbnb Analysis** </span>

## Tasks:
### <span style = 'color: cornflowerblue'> 1. Which region in NYC has the largest number of rentals and the highest prices?</span>
### <span style = 'color: cornflowerblue'> 2. How neighborhoods and amenities influence Airbnb prices?</span>
### <span style = 'color: cornflowerblue'> 3. Which borough of NYC has the highest price range?</span>
### <span style = 'color: cornflowerblue'> 4. Which period of time in a year has the largest number of rentals?</span>


### <span style = 'color: brown'> Import required packages </span>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### <span style = 'color: brown'> Import Data and Inpsect </span>



In [2]:
listings = pd.read_csv('./listings.csv')
reviews = pd.read_csv('./reviews.csv')
neighbors = pd.read_csv('./neighbourhoods.csv')

<ipython-input-2-c8767423235d>:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  listings = pd.read_csv('./listings.csv')


#### Listing Data

In [3]:
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1,NaN
1,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,365,0,NaN
2,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.72,1,0,0,NaN
3,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,575,2023-02-19,3.41,1,106,52,NaN
4,5136,"Large Sunny Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1,NaN


In [4]:
listings.shape

(42931, 18)

In [5]:
print(f'Duplicated Rows: {listings.duplicated().sum()}')
print(f'Total Null Values in the dataset: {listings.isnull().sum().sum()}')

Duplicated Rows: 0
Total Null Values in the dataset: 63555


In [6]:
listings.isnull().sum()

id                                    0
name                                 12
host_id                               0
host_name                             5
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10304
reviews_per_month                 10304
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                           42930
dtype: int64

##### **Some listings are missing name, host_name, last_review, reviews_per_month, and license. In this case, I will drop all these columns because they are not really relevant to the topic of analysis. and a few other columns will be dropped for convenience.**

In [7]:
listing_data = listings.drop(columns = ['name', 'host_id', 'host_name','license'], axis = 1).copy()
listing_data.isnull().sum()

id                                    0
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10304
reviews_per_month                 10304
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
dtype: int64

##### **Because last_review and reviews_per_month are important features when determining whether the listing is popular, therefore, those two columns will be kept and null values will be filled with randomized date from the earliest date to the latest date in that column. And reviews_per_month will be predicted through machine learning for better analysis.**

#### Reviews Data

In [8]:
reviews.head()

,listing_id,date
0,2595,2009-11-21
1,2595,2009-12-05
2,2595,2009-12-10
3,2595,2010-04-09
4,2595,2010-05-25


In [9]:
reviews.shape

(1110024, 2)

In [10]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110024 entries, 0 to 1110023
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1110024 non-null  int64 
 1   date        1110024 non-null  object
dtypes: int64(1), object(1)
memory usage: 16.9+ MB


### Neighbors Data

In [11]:
neighbors.head()

,neighbourhood_group,neighbourhood
0,Bronx,Allerton
1,Bronx,Baychester
2,Bronx,Belmont
3,Bronx,Bronxdale
4,Bronx,Castle Hill


In [12]:
neighbors.shape

(230, 2)

In [13]:
neighbors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   neighbourhood_group  230 non-null    object
 1   neighbourhood        230 non-null    object
dtypes: object(2)
memory usage: 3.7+ KB


### <span style = 'color: brown'> Analysis </span>

In [14]:
listing_data.head()

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,2595,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1
1,5121,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,365,0
2,5203,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.72,1,0,0
3,5178,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,575,2023-02-19,3.41,1,106,52
4,5136,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1


#### Merge the reviews table with the listings table with number of reviews from 2022-06-01 to now.

In [15]:
reviews.head()
reviews['date'] = pd.to_datetime(reviews['date'])

In [16]:
reviews_22_23 = reviews[(reviews['date'] >= '2022-06-01') & (reviews['date'] < '2023-06-01')]

# Split date into seasons - According to Meteorological Seasons
# Summer: 2022-06-01 to 2022-08-31
# Fall: 2022-09-01 to 2022-11-30
# Winter: 2022-12-01 to 2023-02-28
# Spring: 2023-03-01 to 2023-05-31

reviews_22_23['season'] = reviews_22_23['date'].dt.month%12 // 3 + 1
reviews_22_23['month'] = reviews_22_23['date'].dt.month_name()
reviews_22_23

<ipython-input-16-84026badb2be>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_22_23['season'] = reviews_22_23['date'].dt.month%12 // 3 + 1
<ipython-input-16-84026badb2be>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_22_23['month'] = reviews_22_23['date'].dt.month_name()


,listing_id,date,season,month
48,2595,2022-06-21,3,June
68,5136,2022-08-10,3,August
357,5586,2022-07-16,3,July
358,5586,2022-08-29,3,August
359,5586,2022-10-03,4,October
...,...,...,...,...
1110019,832030468828225424,2023-03-04,2,March
1110020,832045240627733621,2023-03-04,2,March
1110021,832045240627733621,2023-03-05,2,March
1110022,832049090494847039,2023-03-01,2,March


In [17]:
reviews_22_23['date'].dt.month.unique()

array([ 6,  8,  7, 10,  9, 12,  1,  2, 11,  3], dtype=int64)

In [18]:
reviews_22_23['season'] = reviews_22_23['season'].replace({
    1: 'Winter',
    2: 'Spring',
    3: 'Summer',
    4: 'Fall'
})

<ipython-input-18-c4c9db01848e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_22_23['season'] = reviews_22_23['season'].replace({


In [19]:
reviews_22_23.head()

,listing_id,date,season,month
48,2595,2022-06-21,Summer,June
68,5136,2022-08-10,Summer,August
357,5586,2022-07-16,Summer,July
358,5586,2022-08-29,Summer,August
359,5586,2022-10-03,Fall,October


In [20]:
number_reviews_22_23 = pd.DataFrame(reviews_22_23.groupby(['listing_id'])['date'].count()).reset_index()
number_reviews_22_23.rename(columns = {'date': '#reviews_22_to_23'}, inplace=True)
number_reviews_22_23.head()

,listing_id,#reviews_22_to_23
0,2595,1
1,5136,1
2,5178,38
3,5586,3
4,5803,11


In [21]:
season_reviews_22_23 = pd.DataFrame(reviews_22_23.groupby(['season'])['listing_id'].count()).reset_index()
season_reviews_22_23.rename(columns = {'listing_id': '#reviews'}, inplace = True)

In [22]:
month_reviews_22_23 = pd.DataFrame(reviews_22_23.groupby(['month'])['listing_id'].count()).reset_index()
month_reviews_22_23.rename(columns = {'listing_id': '#reviews'}, inplace = True)

In [23]:
month_reviews_22_23

,month,#reviews
0,August,29511
1,December,32922
2,February,18475
3,January,24957
4,July,27832
5,June,27280
6,March,1558
7,November,31080
8,October,35771
9,September,34017


In [24]:
extra_rows = pd.DataFrame({'month': ['April','May'], '#reviews': [0, 0]}, index = [10, 11])
month_reviews_22_23_revised = pd.concat([month_reviews_22_23, extra_rows])


In [25]:
data = listing_data.merge(number_reviews_22_23, how = 'left', left_on = 'id', right_on = 'listing_id')
data.head()

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,listing_id,#reviews_22_to_23
0,2595,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1,2595.0,1.0
1,5121,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,365,0,NaN,NaN
2,5203,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.72,1,0,0,NaN,NaN
3,5178,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,575,2023-02-19,3.41,1,106,52,5178.0,38.0
4,5136,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1,5136.0,1.0


In [26]:
data['#reviews_22_to_23'].fillna(0, inplace = True)
data.head()

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,listing_id,#reviews_22_to_23
0,2595,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1,2595.0,1.0
1,5121,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,365,0,NaN,0.0
2,5203,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.72,1,0,0,NaN,0.0
3,5178,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,575,2023-02-19,3.41,1,106,52,5178.0,38.0
4,5136,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1,5136.0,1.0


In [27]:
data.drop(columns = ['listing_id'], inplace = True)

In [28]:
data.head()

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,#reviews_22_to_23
0,2595,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1,1.0
1,5121,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,365,0,0.0
2,5203,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.72,1,0,0,0.0
3,5178,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,575,2023-02-19,3.41,1,106,52,38.0
4,5136,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1,1.0


#### Explore relationships between features
##### Room_type

In [29]:
data['room_type'].value_counts()

Entire home/apt    24279
Private room       17879
Shared room          576
Hotel room           197
Name: room_type, dtype: int64

In [30]:
data.groupby(['room_type'])['number_of_reviews'].mean().sort_values(ascending=False)

room_type
Hotel room         52.243655
Entire home/apt    26.591087
Private room       24.732200
Shared room        20.729167
Name: number_of_reviews, dtype: float64

In [31]:
data.groupby(['room_type'])['#reviews_22_to_23'].mean().sort_values(ascending=False)

room_type
Hotel room         6.944162
Entire home/apt    6.475802
Private room       5.735668
Shared room        3.925347
Name: #reviews_22_to_23, dtype: float64

In [32]:
data.groupby(['room_type'])['price'].mean().sort_values(ascending=False)

room_type
Hotel room         309.959391
Entire home/apt    249.255365
Private room       135.015046
Shared room        126.250000
Name: price, dtype: float64

###### **Room type of entire home/apt and private room has the largest number of listings, however, hotel rooms are more popular these days and even though hotel rooms have higher price range, but their number of reviews overall and for last year are the highest among all room type.**

##### Neighborhoods

In [33]:
data.groupby(['neighbourhood_group'])['id'].count().sort_values(ascending = False)

neighbourhood_group
Manhattan        17658
Brooklyn         16237
Queens            6916
Bronx             1691
Staten Island      429
Name: id, dtype: int64

In [34]:
data.groupby(['neighbourhood_group'])['price'].mean().sort_values(ascending=False)

neighbourhood_group
Staten Island    309.037296
Manhattan        268.365783
Brooklyn         162.766829
Queens           128.173655
Bronx            117.512123
Name: price, dtype: float64

In [35]:
data.groupby(['neighbourhood_group'])['number_of_reviews'].mean().sort_values(ascending=False)

neighbourhood_group
Staten Island    35.452214
Queens           30.124928
Brooklyn         28.739484
Bronx            25.456535
Manhattan        21.337694
Name: number_of_reviews, dtype: float64

In [36]:
data.groupby(['neighbourhood_group'])['#reviews_22_to_23'].mean().sort_values(ascending=False)

neighbourhood_group
Staten Island    8.832168
Queens           8.596588
Bronx            7.519811
Brooklyn         6.342674
Manhattan        4.782988
Name: #reviews_22_to_23, dtype: float64

###### **Staten Island is the most popular place for airbnb even though it has the lowest number of listings and its price is the highest among all neighbours. And it has the highest number of reviews over years. Queens is the second most popular place just right behind Staten Island.**

### <span style = 'color: brown'> Output data for visualizations in Tableau </span>

In [37]:
data.to_csv('airbnb_nyc.csv')
season_reviews_22_23.to_csv('season_data_airbnb_nyc.csv')
month_reviews_22_23_revised.to_csv('month_data_airbnb_nyc_revised.csv') 

### <span style= 'color:brown'> Tableau Visualizations </span>

In [38]:
%%html
<div class='tableauPlaceholder' id='viz1687967640743' style='position: relative'><noscript><a href='#'><img alt=' Airbnb NYC Analysis ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ai&#47;AirbnbNYCAnalysisv2&#47;AirbnbNYCAnalysis&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AirbnbNYCAnalysisv2&#47;AirbnbNYCAnalysis' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ai&#47;AirbnbNYCAnalysisv2&#47;AirbnbNYCAnalysis&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1687967640743');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1366px';vizElement.style.height='795px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1366px';vizElement.style.height='795px';} else { vizElement.style.width='100%';vizElement.style.height='2227px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>